In [7]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [8]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [9]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:こんにちは！'

こんにちは！どんなことをお手伝いできますか？


'質問:東北6県は？'

東北地方は日本の地域で、以下の6つの県で構成されています：

1. 青森県（あおもりけん）
2. 岩手県（いわてけん）
3. 宮城県（みやぎけん）
4. 秋田県（あきたけん）
5. 山形県（やまがたけん）
6. 福島県（ふくしまけん）

この6県が東北地方の主要な県です。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県のお土産には多くの魅力的な商品があります。以下は、特におすすめのお土産のいくつかです。

1. **仙台名物牛タン** - 素材にこだわった本物の牛タンは、宮城を代表するグルメです。お土産用に加工されたものも多く販売されています。

2. **ずんだ餅** - 枝豆を使ったあんこが特徴のずんだ餅は、甘さと豆の風味が楽しめるお菓子です。

3. **萩の月** - しっとりとしたスポンジケーキで、クリームを包み込んだセンスあるスイーツです。とても人気があります。

4. **くるみゆべし** - 濃厚なくるみの風味と、甘い味わいの和菓子。外はモチっとした食感で、内側にはくるみが香ります。

5. **地酒** - 宮城県は多くの酒蔵があり、地元の米を使った日本酒はお酒好きにはたまらないお土産です。

6. **三陸の海産物** - 牡蠣やいくら、海藻など新鮮な海産物は、食卓を華やかに彩るアイテムです。

これらの情報は、宮城県の公式サイトや観光ガイドなどからもチェックできます。お土産を選ぶ際は、地元の特産品を意識すると良いでしょう。詳しい情報はぜひ、各リンクを参考にしてください。

- [宮城県物産振興協会](https://ec.miyagibussan.or.jp/)
- [じゃらんニュース](https://www.jalan.net/news/article/93858/)
- [旬仙たんぼ](https://shunsentanbou.pref.miyagi.jp/theme/miyage/)
- [東北スタンダード](https://tohokuru.jp/collections/miyagi?srsltid=AfmBOoqc6oOQmtyOg5WeYJ9kqp9FIpW1f4df1kywX6Hw6-m_cZN1FuOk)

この情報を参考にして、お土産選びを楽しんでください！

---ご利用ありがとうございました！---
